In [1]:
import sys

import time
import pandas as pd
import numpy as np


#   paths
ANKI_PATH = 'D:/Exams/Jupyter Notebooks/Anki Files/anki-master'
COLLECTION_PATH = "D:/Exams/Jupyter Notebooks/Anki Files//collection.anki2"
pd.options.display.float_format = '{:.0f}'.format
sys.path.append(ANKI_PATH)

import anki
import math
from anki import Collection
from anki.utils import intTime


In [2]:
col = Collection(COLLECTION_PATH)

cards = col.db.all("select id,nid,did from cards")

col.close()

In [3]:
col = Collection(COLLECTION_PATH)

card_id = []
note_id = []
tags =[]

for card in cards:
    card_id.append(card[0])
    note_id.append(card[1])
    tags.append(col.db.all("select tags from notes where id = " + str(card[1]))[0][0].split())
    
cards_w_tags = pd.DataFrame(list(zip(card_id, note_id,tags)), 
            columns =['Card_ID', 'Note_ID','Tags'])

col.close()

In [4]:
col = Collection(COLLECTION_PATH)

revlog = col.db.all("select id,cid,ease from revlog")

col.close()

In [5]:
date = []
card_id = []
ease =[]

result=[]
for card in revlog:
    date.append(time.strftime('%Y-%m-%d', time.localtime(card[0]/1000)))
    card_id.append(card[1])
    ease.append(card[2])

revlog_cards = pd.DataFrame(list(zip(date, card_id,ease)), 
            columns =['Date', 'Card_ID','Ease'])
revlog_cards = revlog_cards[revlog_cards['Date'] >= '2018-01-01'] 

In [6]:
revlog_cards = pd.merge(revlog_cards, cards_w_tags, how='left', on=['Card_ID'])

In [7]:
#revlog_cards[pd.isnull(revlog_cards).any(axis=1)]
revlog_cards = revlog_cards[np.isfinite(revlog_cards['Note_ID'])]

In [8]:
def f(row):
    if row['Ease'] < 3:
        val = 0
    else:
        val = 1
    return val

revlog_cards['Correct'] = revlog_cards.apply(f, axis=1)

In [9]:
max_len = 0
index = 0
for item in revlog_cards['Tags']:
    if len(item) > max_len:
        max_len = len(item)
    index = index+1


In [10]:
revlog_cards[['Tag1','Tag2','Tag3','Tag4']] = pd.DataFrame(
    revlog_cards.Tags.values.tolist(), index= revlog_cards.index)

In [11]:
 def exam(row):
    if any("Exam7" in s for s in row['Tags']):
        val = 'Exam7'
    elif any("Exam8" in s for s in row['Tags']):
        val = 'Exam8'
    elif any("Exam9" in s for s in row['Tags']):
        val = 'Exam9'
    else:
        val = 'Other'
    return val

revlog_cards['Exam'] = revlog_cards.apply(exam, axis=1)

Tag_list = revlog_cards['Tag1'].unique().tolist() + revlog_cards['Tag2'].unique().tolist() + revlog_cards[
    'Tag3'].unique().tolist() + revlog_cards['Tag4'].unique().tolist()
Tag_list = list(set(Tag_list))

In [28]:
from bokeh.layouts import row, column
from bokeh.models import Slider, Select, DatetimeTickFormatter, HoverTool
from bokeh.models.widgets import DateSlider
from bokeh.plotting import figure, output_file, show
from bokeh.io import curdoc
from bokeh.palettes import Dark2_5 as palette
import itertools

In [35]:
from datetime import date, timedelta

#exam_select = Select(value='Exam8', title='City', options=['Exam7','Exam8','Exam9'])
#ending_date = DateRangeSlider(title="Ending Date", value=datetime.today().strftime('%Y-%m-%d'),
#                     start='2018-01-01', end=datetime.today().strftime('%Y-%m-%d'))

#ending_date_slider = DateSlider(title="Ending Date", start=date(2019, 1, 1), end=date.today(),
#                                value=date.today(), step=1)
max_days = Slider(title='Lookback period',start = 0,end=200,value=10, step=1)


#Setting up x and y data
datelist = pd.date_range(start='7-3-2019',end=date.today(), freq='D')

data_subset = revlog_cards[revlog_cards['Exam'] == 'Exam8']
tags = pd.unique(data_subset[['Tag1', 'Tag2','Tag3','Tag4']].values.ravel('K'))
tags = [x for x in tags if x != None]

acc_val = []
acc_tag = []
acc_date = []

for date in datelist:
    for tag in tags:
        data = data_subset[(data_subset['Date'] > (date - timedelta(days=max_days.value)).strftime('%Y-%m-%d')) 
                           & (data_subset['Date'] <= date.strftime('%Y-%m-%d'))]
        data = data[data['Tag1'].str.contains(tag) | data['Tag2'].str.contains(tag) |
           data['Tag3'].str.contains(tag) | data['Tag4'].str.contains(tag)]
        
        acc_date.append(date)
        acc_tag.append(tag)
        acc_val.append(data['Correct'].sum()/len(data))

final = pd.DataFrame(list(zip(acc_date, acc_tag,acc_val)), columns =['Date', 'Tag','Value'])



#TOOLTIPS = [
#    ('Paper', "$name"),
#    ('Accuracy', '@y'),
#    ('Date', "@x{%Y-%m-%d}"),
   # ("y", "$y"),
   # ("value", "@image"),
   # ('squared', '@squared')
#    ]




# Set up plot
plot = figure(plot_height=900, plot_width=1500, title="Topic Accuracy",
              tools="pan,reset,wheel_zoom,box_zoom",
              y_range=[-0.1, 1.2])



#colors has a list of colors which can be used in plots 
colors = itertools.cycle(palette)

for tag in tags:
    plot.line(datelist, final[final['Tag'] == tag]['Value'], 
              line_width=2, line_alpha=0.6,legend=tag,color=next(colors), name = tag)

plot.legend.location = "top_left"
plot.legend.click_policy="hide"
plot.xaxis.formatter = DatetimeTickFormatter(days=['%m/%d'])


# Hover tool
plot.add_tools(HoverTool(
    tooltips=[
        ('Paper', '$name'), 
        ('Accuracy', '@y{%0.4}'),
        ('Date', '@x{%F}')
    ],
    formatters={'x': 'datetime'}
))



inputs = row(column(max_days),plot)

show(inputs)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in longlong_scalars
